In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vocab-dict-final/vocab_dict_v2/dataset_dict.json
/kaggle/input/vocab-dict-final/vocab_dict_v2/validation/state.json
/kaggle/input/vocab-dict-final/vocab_dict_v2/validation/dataset_info.json
/kaggle/input/vocab-dict-final/vocab_dict_v2/validation/data-00000-of-00001.arrow
/kaggle/input/vocab-dict-final/vocab_dict_v2/train/state.json
/kaggle/input/vocab-dict-final/vocab_dict_v2/train/dataset_info.json
/kaggle/input/vocab-dict-final/vocab_dict_v2/train/data-00000-of-00001.arrow


In [3]:
!pip install datasets
!pip install -q -U google-generativeai

In [4]:
import os
import time
import wandb
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
from torch.utils import data
from datasets import load_dataset
from transformers import GPT2TokenizerFast
from kaggle_secrets import UserSecretsClient
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.generativeai.types import RequestOptions
import google.generativeai as gemini_ai
from google.api_core import retry
import json
from collections import Counter
import random
import nltk
import re
from datasets import Dataset, DatasetDict
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers
from transformers import PreTrainedTokenizerFast
import torch.nn.functional as F

In [5]:
user_secrets = UserSecretsClient()

In [6]:
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

In [7]:
wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
gemini_api_key = user_secrets.get_secret("GEMINI_API_KEY")

In [9]:
gemini_ai.configure(api_key=gemini_api_key)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
config = {
    "BLOCK_SIZE": 128,
    "EMB_SIZE": 256,
    "N_ATTENTION_HEADS": 8,
    "N_DECODER_BLOCKS": 4,
    "VOCAB_SIZE": 10000,
    "MAX_OUT_TOKENS": 200,
    "EVAL_ITER": 100,
    "LR": 3e-4,
    "BATCH_SIZE": 32,
    "EPOCHS": 5,
    "PATIENCE": 3,
    "MODEL_NAME": "custom-8M",
    "WORKING_DIR": "/kaggle/working",
    "DEVICE": 'cuda' if torch.cuda.is_available() else 'cpu'
}
assert config['EMB_SIZE'] % config['N_ATTENTION_HEADS'] == 0

In [12]:
wandb.init(
    project='custom-8M',
    config=config
)
text_table = wandb.Table(columns=['epoch', 'loss', 'predicted text'])

wandb: Currently logged in as: siddharthdhara17 (dhara). Use `wandb login --relogin` to force relogin


In [13]:
load_model = False

In [14]:
load_df = False

In [15]:
dataset = load_dataset("roneneldan/TinyStories")

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [16]:
loaded_vocab_dataset = DatasetDict.load_from_disk('/kaggle/input/vocab-dict-final/vocab_dict_v2')

In [17]:
custom_vocab = loaded_vocab_dataset['train']['word']
new_vocab_size = len(custom_vocab)
print(new_vocab_size)

9996


In [18]:
custom_vocab_dict = {word: idx for idx, word in enumerate(custom_vocab)}
if "[UNK]" not in custom_vocab_dict:
    print("Adding [UNK] token to the vocabulary.")
    custom_vocab_dict["[UNK]"] = len(custom_vocab_dict)

Adding [UNK] token to the vocabulary.


In [19]:
used_dataset_size = 100000

In [20]:
sampled_dataset = dataset['train'].train_test_split(train_size=0.8, test_size=0.2)
train_dataset, val_dataset = sampled_dataset['train'].select(range(int(0.8 * used_dataset_size))), sampled_dataset['test'].select(range(int(0.2 * used_dataset_size)))

In [21]:
class GPT2FromScratch(nn.Module):
    def __init__(self, config):
        super(GPT2FromScratch, self).__init__()
        self.embeddings = nn.Embedding(config["VOCAB_SIZE"], config["EMB_SIZE"])
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=config["EMB_SIZE"],
                nhead=config["N_ATTENTION_HEADS"],
                dim_feedforward=config["EMB_SIZE"] * 4,
                activation='gelu'
            )
            for _ in range(config["N_DECODER_BLOCKS"])
        ])
        self.final_norm = nn.LayerNorm(config["EMB_SIZE"])
        self.head = nn.Linear(config["EMB_SIZE"], config["VOCAB_SIZE"])
        self.block_size = config.get("BLOCK_SIZE", 128)  # Define block size for context

    def forward(self, x):
        x = self.embeddings(x)
        for block in self.blocks:
            x = block(x)
        x = self.final_norm(x)
        return self.head(x)

    def generate(self, idx, max_new_tokens):
        self.eval()  # Ensure the model is in evaluation mode
        for _ in range(max_new_tokens):
            # Crop idx to the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # Get the predictions
            logits = self(idx_cond)  # Only use logits (ignore loss)
            # Focus only on the last time step
            logits = logits[:, -1, :]  # (B, VOCAB_SIZE)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, VOCAB_SIZE)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append the sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


In [22]:
# # LoRA Layer Integration for Low-Rank Adaptation on the final layer

# class LoRA(nn.Module):
#     def __init__(self, layer, rank=4):
#         super(LoRA, self).__init__()
#         self.layer = layer
#         self.rank = rank
#         self.lora_A = nn.Linear(layer.in_features, rank, bias=False)
#         self.lora_B = nn.Linear(rank, layer.out_features, bias=False)
#         nn.init.normal_(self.lora_A.weight, std=0.02)
#         nn.init.normal_(self.lora_B.weight, std=0.02)
        
#     def forward(self, x):
#         return self.layer(x) + self.lora_B(self.lora_A(x))

In [23]:
# class GPT2WithLoRA(nn.Module):
#     def __init__(self, config):
#         super(GPT2WithLoRA, self).__init__()
#         self.model = GPT2FromScratch(config)
#         self.model.head = LoRA(self.model.head)

#     def forward(self, x):
#         logits = self.model(x)
#         return {'logits': logits}  # Return a dictionary with logits

#     def generate(self, input_ids, max_length=50, **kwargs):
#         output = input_ids
#         for _ in range(max_length):
#             logits = self.forward(output)['logits']
#             next_token = torch.argmax(logits[:, -1], dim=-1).unsqueeze(-1)
#             output = torch.cat((output, next_token), dim=1)
#         return output


In [24]:
# Create a tokenizer from scratch with custom vocab
tokenizer = Tokenizer(models.WordLevel(vocab=custom_vocab_dict, unk_token="[UNK]"))
base_tokenizer = AutoTokenizer.from_pretrained("roneneldan/TinyStories-1Layer-21M")
# Set up pre-tokenizer, normalizer, and decoder (as used in most tokenizers)
tokenizer.normalizer = normalizers.Lowercase()
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.decoder = decoders.WordPiece()

# Save the tokenizer to a file
tokenizer.save("custom_tokenizer.json")

# Load this tokenizer into a PreTrainedTokenizerFast
custom_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="custom_tokenizer.json",
    model_max_length=base_tokenizer.model_max_length
)

# Add special tokens if needed
custom_tokenizer.add_special_tokens({'additional_special_tokens': ['<sos>', '<eos>']})
custom_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# Save the custom tokenizer
custom_tokenizer.save_pretrained("custom_tokenizer")

# Reload and print the vocabulary size to confirm
custom_tokenizer = AutoTokenizer.from_pretrained("custom_tokenizer")
print(f"Custom tokenizer vocabulary size: {custom_tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Custom tokenizer vocabulary size: 9997


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# Tokenization function for HuggingFace dataset
def tokenize_function(examples):
    return custom_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=config['BLOCK_SIZE'])

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [26]:
# Convert tokenized dataset to PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids'])
val_dataset.set_format(type='torch', columns=['input_ids'])

train_loader = data.DataLoader(train_dataset, batch_size=config['BATCH_SIZE'], shuffle=True)
val_loader = data.DataLoader(val_dataset, batch_size=config['BATCH_SIZE'], shuffle=False)

In [27]:
print(len(custom_tokenizer))

10000


In [28]:
model = GPT2FromScratch(config)

In [29]:
model = model.to(config['DEVICE'])

In [30]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

Total Parameters: 8289552


In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=config['LR'])
loss_fn = torch.nn.CrossEntropyLoss()

In [32]:
@torch.no_grad()
def eval_model(training_model: torch.nn.Module, val_loader: data.DataLoader):
    training_model.eval()
    losses = torch.zeros(config['EVAL_ITER'])
    for k in range(config['EVAL_ITER']):
        batch = next(iter(val_loader))
        s_val = batch['input_ids'].to(config['DEVICE'])
        t_val = s_val[:, 1:].clone()
        s_val = s_val[:, :-1]
        val_logits = training_model(s_val)
        val_logits = val_logits.view(s_val.size(0) * s_val.size(1), config['VOCAB_SIZE'])
        t_val = t_val.view(s_val.size(0) * s_val.size(1))
        losses[k] = torch.nn.functional.cross_entropy(val_logits, t_val).item()
    training_model.train()
    return losses.mean()

In [33]:
def train_model(model, train_loader, val_loader, optimizer, config, loss_fn):
    best_val_loss = float('inf')
    patience_counter = 0
    
    try:
        for epoch in range(config['EPOCHS']):
            model.train()
            epoch_loss = 0
            epoch_progress = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{config['EPOCHS']}: ", leave=False)
            
            for b_idx, batch in enumerate(epoch_progress):
                sources = batch['input_ids'].to(config['DEVICE'])
                targets = sources[:, 1:].clone().to(config['DEVICE'])
                sources = sources[:, :-1]
                logits = model(sources)  # Direct tensor output
                batch_size = sources.size(0)
                seq_length = sources.size(1)
                logits = logits.view(batch_size * seq_length, config['VOCAB_SIZE'])
                targets = targets.view(batch_size * seq_length)
                loss = loss_fn(logits, targets)
                wandb.log({"loss": loss.item()})
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                avg_loss = epoch_loss / (b_idx + 1)
                epoch_progress.set_postfix(training_loss=avg_loss)

            avg_epoch_loss = epoch_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{config['EPOCHS']} completed with average training loss: {avg_epoch_loss}")
            
            val_loss = eval_model(model, val_loader)
            print(f"Validation loss after {epoch+1} epochs: {val_loss}")
            wandb.log({"val_loss": val_loss})

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                print(f"New best validation loss: {val_loss}.")
            else:
                patience_counter += 1
                print(f"No improvement in validation loss. Patience counter: {patience_counter}")
                
            if patience_counter >= config['PATIENCE']:
                print("Early stopping triggered.")
                break
            
    except KeyboardInterrupt:
        print("Training interrupted.")
    print("Training completed.")

In [34]:
if not load_model:
    train_model(model, train_loader, val_loader, optimizer, config, loss_fn)

Epoch 1/5 completed with average training loss: 3.6751812463760376
Validation loss after 1 epochs: 3.4175243377685547
New best validation loss: 3.4175243377685547.


Epoch 2/5 completed with average training loss: 3.464756632709503
Validation loss after 2 epochs: 3.3774402141571045
New best validation loss: 3.3774402141571045.


Epoch 3/5 completed with average training loss: 3.43430044965744
Validation loss after 3 epochs: 3.3512024879455566
New best validation loss: 3.3512024879455566.


Epoch 4/5 completed with average training loss: 3.4177992327690125
Validation loss after 4 epochs: 3.349935293197632
New best validation loss: 3.349935293197632.


Epoch 5/5 completed with average training loss: 3.4072976881980894
Validation loss after 5 epochs: 3.3435232639312744
New best validation loss: 3.3435232639312744.
Training completed.


In [35]:
model_req_path = config['WORKING_DIR']+'/model'

In [36]:
if not load_model:
    if not os.path.exists(model_req_path):
        os.mkdir(model_req_path)
        
    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
                }, model_req_path+'/'+config['MODEL_NAME']+'.pt')

    print("Model saved!")

Model saved!


In [37]:
if load_model and os.path.exists(model_req_path):
    checkpoint = torch.load(model_req_path+'/'+config['MODEL_NAME']+'.pt', weights_only=True)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print("Loaded the model!")
elif not os.path.exists(model_req_path):
    print("Model directory not found! Please check the path.")

In [38]:
def prepare_input(text, tokenizer, device, block_size=128):
    # Tokenize and encode the input text
    inputs = tokenizer(
        text, return_tensors="pt", padding="max_length",
        truncation=True, max_length=block_size
    )
    # Move input tensors to the appropriate device
    return {key: val.to(device) for key, val in inputs.items()}

In [39]:
def generate_text(model, tokenizer, input_text, config):
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(config["DEVICE"])
    
    # Generate text using the model's `generate` method
    output_ids = model.generate(
        input_ids,
        max_new_tokens=config['MAX_OUT_TOKENS']  # Max tokens to generate
    )
    
    # Decode the generated IDs to text
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text


In [50]:
def evaluate_text_gemini(generated_text):
    # Use the generative model directly for evaluation
    model2 = gemini_ai.GenerativeModel("gemini-1.5-flash")
    
    # Generate content and get the response
    response = model2.generate_content(generated_text, request_options=RequestOptions(retry=retry.Retry(initial=10, multiplier=2, maximum=60, timeout=500)))
    
    # Check if response contains valid content
    if response.candidates:
        eval_response = response.candidates[0].content.parts[0].text  # Get the text of the first candidate
        return eval_response
    else:
        print("No valid candidates in the response. Check the generated text or API settings.")
        return None

In [45]:
input_texts_list = [
    "In a bustling city filled with secrets, a shadow loomed.",
    "High in the mountains, a lone traveler braved the storm.",
    "Beneath the waves, in a hidden underwater kingdom, life thrived.",
    "It was a quiet night until the distant howls broke the silence.",
    "In a world where dragons flew free, danger was never far.",
    "On a small farm, a young girl discovered a mysterious egg.",
    "In a town where everyone whispered, the new stranger caused a stir.",
    "Under the light of the full moon, something magical began to stir.",
    "In a time of peace, a hidden evil began to rise.",
    "Amidst the golden sands of the desert, a lost caravan wandered.",
    "In the heart of the enchanted forest, a hidden village thrived.",
    "Aboard a ship sailing unknown seas, the crew faced a peculiar sight.",
    "In a world where animals spoke, a young boy sought adventure.",
    "Deep within the icy tundra, an ancient secret lay buried.",
    "On a stormy night, a stranger knocked at the castle door.",
    "In a village plagued by mysteries, a young detective took charge.",
    "Across the galaxy, explorers marveled at a new world.",
    "Underneath the quiet streets, a hidden society had formed.",
    "Long ago, a powerful wizard disappeared without a trace.",
    "At the edge of the world, a brave crew faced the unknown.",
    "In a city that never slept, two souls crossed paths unexpectedly.",
    "In a school for magical creatures, new students arrived.",
    "Hidden in the clouds, a floating kingdom kept its secrets.",
    "Far in the distant future, humanity encountered its first alien.",
    "Under a blanket of stars, two friends made a promise.",
    "In a library of forgotten books, a mysterious journal appeared.",
    "Aboard a train that never stopped, secrets unraveled slowly.",
    "Amid a sea of stars, a lone spaceship drifted in silence.",
    "In a kingdom of snow and ice, a prophecy was foretold.",
    "Deep in the jungle, explorers discovered a glowing stone.",
    "In a village under a curse, a hero was born.",
    "At the dawn of time, the first humans encountered magic.",
    "On a distant moon, an outpost signaled for help.",
    "In a quiet town, every night brought new mysteries.",
    "Beneath the great pyramids, an ancient secret was uncovered.",
    "In a world without color, a single red flower bloomed.",
    "On a ship lost at sea, whispers of an island spread.",
    "In the middle of nowhere, a door to another world appeared.",
    "In a castle of mirrors, reflections began to act strangely.",
    "On the edge of a cliff, a young prince made a fateful decision.",
    "In a forest where time stood still, a visitor arrived.",
    "In a kingdom ruled by animals, a lion declared his rule.",
    "In a world where wishes came true, a girl wished for more time.",
    "In a school where shadows came alive, mysteries abounded.",
    "In a library that seemed endless, a strange book was found.",
    "On a small island, villagers began to notice odd happenings.",
    "In the great desert, a treasure was hidden for centuries.",
    "In a city beneath the earth, a new ruler emerged.",
    "In a world divided by seasons, an eternal summer began.",
    "At the top of a mountain, a temple held the key to truth.",
    "In a town where time rewound each day, mysteries deepened.",
    "On a snowy peak, two climbers discovered an ancient statue.",
    "In a mansion where paintings moved, a mystery unraveled.",
    "At the crossroads of realms, two adventurers met.",
    "In a school hidden in the woods, every student had a secret.",
    "In a town of endless rain, hope was a rare sight.",
    "On a train bound for nowhere, strange passengers arrived.",
    "In a forest where trees whispered, a path emerged.",
    "In a world where stars guided destiny, a comet foretold change.",
    "In a lonely tower, a forgotten sorceress waited.",
    "At the edge of a lake, the reflection showed another world.",
    "In a world beneath the clouds, legends of the sky spread.",
    "In a kingdom of night, a lone warrior sought dawn.",
    "In a garden of eternal flowers, time stood still.",
    "On an island that disappeared each night, a story began.",
    "In a city that glittered like gold, shadows lurked.",
    "In a land where dreams came alive, a nightmare was born.",
    "On the longest night, a hero's journey began.",
    "In a kingdom lost to time, an old legend resurfaced.",
    "In a forest cloaked in fog, paths led nowhere.",
    "In a small shop, a mysterious item granted wishes.",
    "On the day the sun didn't rise, fear spread.",
    "In a town with no maps, wanderers were welcome.",
    "Under a sky of falling stars, two souls met.",
    "In a house with endless rooms, a mystery unraveled.",
    "In a town where no one aged, secrets were kept.",
    "In the middle of the ocean, a floating castle appeared.",
    "At the heart of the desert, a lone tree bloomed.",
    "In a world ruled by music, silence was feared.",
    "On the night of the festival, a strange guest arrived.",
    "In a land where darkness ruled, a light began to shine.",
    "In a city where everyone wore masks, truths hid.",
    "In a world of whispers, silence was a power.",
    "On the eve of battle, a hero was forged.",
    "In a castle of glass, a kingdom looked on.",
    "In the kingdom of echoes, a voice was heard.",
    "In a forest where dreams came true, nightmares hid.",
    "In a town with endless winters, a new day dawned.",
    "In a realm where seasons changed daily, stories grew.",
    "Under the gaze of ancient gods, mortals lived.",
    "In a world frozen in time, a clock began to tick.",
    "In a library of the lost, an old tale was read.",
    "In a meadow under starlight, two friends found magic.",
    "In a city where clocks ran backward, futures changed.",
    "In a kingdom ruled by children, a new game began.",
    "In a land where the moon never rose, stars told tales.",
    "In the far north, where the aurora danced, legends lived.",
    "At the edge of eternity, two lovers met.",
    "In a village of music, silence brought fear.",
    "In a world where memories could be traded, one boy remembered."
]

In [53]:
pattern = r"Grammar: (\d+)/10; Consistency: (\d+)/10; Creativity: (\d+)/10; Plot: (\d+)/10; Age group: ([A-Z])"
score_list = []
count = 0

In [54]:
step_1_static = (
    "The following exercise, the student is given the beginning of a story. The student needs to complete it into a full story. "
    "The exercise tests the student’s language abilities and creativity. The symbol * marks the separator between the "
    "prescribed beginning and the student’s completion: "
)

step_2 = (
    "Please provide your general assessment about the part written by the student (the one after the * symbol). "
    "Only give the ratings without description and overall could be omitted"
    "Do not give explainations for the ratings."
    "Give them in one single line, separated by semi-colon."
    "Keep the fields for the output consistent, that is keep all the fields mentioned in the next sentence. "
    "Grammer: ; Consistency: ; Creativity: ; Plot: ; Age group: "
    "Is it grammatically correct? Is it consistent with the beginning of the story? Pay special attention to whether the "
    "student manages to complete the sentence which is split in the middle by the separator *."
)

step_3 = (
    "Now, grade the student’s completion in terms of grammar, creativity, consistency with the story’s beginning and "
    "whether the plot makes sense. Moreover, please provide your best guess of what the age of the student might be, "
    "as reflected from the completion. Choose from possible age groups: A: 3 or under. B: 4-5. C: 6-7. D: 8-9. E: 10-12. F: 13-16. "
    "e.g. Grammar: 8/10; Consistency: 7/10; Creativity: 7/10; Plot: 7/10; Age group: E (10-12)"
)

In [ ]:
if not load_df:
    for input_text in input_texts_list:
        output_text = generate_text(model, custom_tokenizer, input_text, config)
        # print(output_text)
        dynamic_part = f"{input_text} Story begins here:*  {''.join(output_text)}. * The story ends here"
        final_prompt = f"{step_1_static}{dynamic_part}\n{step_2}\n{step_3}"
        gemini_generated_response = evaluate_text_gemini(final_prompt)
        print(gemini_generated_response)
        gemini_generated_response = gemini_generated_response.strip()
        count += 1
        
        # print(f"{input_text}; {gemini_generated_response}")
        match = re.search(pattern, gemini_generated_response)
        if match:
            grammar, consistency, creativity, plot, age_group = match.groups()
            score_list.append([input_text, int(grammar), int(consistency), int(creativity), age_group])
        else:
            score_list.append([input_text, 0, 0, 0, "DNF"])
        print(f"Number of prompts appended: {count}")

Grammar: 2/10; Consistency: 1/10; Creativity: 2/10; Plot: 1/10; Age group: B

Number of prompts appended: 1
Grammar: 2/10; Consistency: 1/10; Creativity: 3/10; Plot: 1/10; Age group: B

Number of prompts appended: 2
Grammar: 2/10; Consistency: 1/10; Creativity: 2/10; Plot: 1/10; Age group: B

Number of prompts appended: 3
Grammar: 2/10; Consistency: 1/10; Creativity: 3/10; Plot: 1/10; Age group: B (4-5)

Number of prompts appended: 4
Grammar: 2/10; Consistency: 1/10; Creativity: 3/10; Plot: 1/10; Age group: B

Number of prompts appended: 5
Grammar: 2/10; Consistency: 1/10; Creativity: 2/10; Plot: 1/10; Age group: B

Number of prompts appended: 6
Grammar: 2/10; Consistency: 1/10; Creativity: 3/10; Plot: 1/10; Age group: B

Number of prompts appended: 7
Grammar: 3/10; Consistency: 2/10; Creativity: 3/10; Plot: 1/10; Age group: B

Number of prompts appended: 8
Grammar: 2/10; Consistency: 1/10; Creativity: 3/10; Plot: 1/10; Age group: B

Number of prompts appended: 9
Grammar: 2/10; Consist

In [ ]:
df = pd.DataFrame(score_list, columns=["Input Prompt", "Grammar", "Creativity", "Consistency", "Age Group"])

In [ ]:
result_req_path = config['WORKING_DIR']+'/result'

In [ ]:
if not load_df:
    if not os.path.exists(result_req_path):
        os.mkdir(result_req_path)
        
    df.to_pickle(result_req_path+'/'+'rating_df_'+config['MODEL_NAME']+'.pkl')

print("Dataframe saved!")

In [ ]:
if load_df and os.path.exists(result_req_path):
    df = pd.read_pickle(result_req_path+'/'+'rating_df_'+config['MODEL_NAME']+'.pkl')
    print("Loaded dataframe!")
elif not os.path.exists(result_req_path):
    print("Result directory not found! Please check the path.")

In [ ]:
df.head()

In [ ]:
df.head(20)

In [ ]:
df.tail()

In [ ]:
df.tail(20)

In [69]:
pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [70]:
print("Modules in GPT2FromScratch model:")
for name, module in model.named_modules():
    print(name)

Modules in GPT2FromScratch model:

embeddings
position_embeddings
blocks
blocks.0
blocks.0.self_attn
blocks.0.self_attn.out_proj
blocks.0.linear1
blocks.0.dropout
blocks.0.linear2
blocks.0.norm1
blocks.0.norm2
blocks.0.dropout1
blocks.0.dropout2
blocks.1
blocks.1.self_attn
blocks.1.self_attn.out_proj
blocks.1.linear1
blocks.1.dropout
blocks.1.linear2
blocks.1.norm1
blocks.1.norm2
blocks.1.dropout1
blocks.1.dropout2
blocks.2
blocks.2.self_attn
blocks.2.self_attn.out_proj
blocks.2.linear1
blocks.2.dropout
blocks.2.linear2
blocks.2.norm1
blocks.2.norm2
blocks.2.dropout1
blocks.2.dropout2
blocks.3
blocks.3.self_attn
blocks.3.self_attn.out_proj
blocks.3.linear1
blocks.3.dropout
blocks.3.linear2
blocks.3.norm1
blocks.3.norm2
blocks.3.dropout1
blocks.3.dropout2
final_norm
head


In [71]:
from peft import LoraConfig, get_peft_model

# Define LoRA configuration for specific attention submodules
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=[
        "blocks.0.self_attn.out_proj",
        "blocks.1.self_attn.out_proj",
        "blocks.2.self_attn.out_proj",
        "blocks.3.self_attn.out_proj",
       
    ],
    bias="none"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 8,192 || all params: 8,559,888 || trainable%: 0.0957


In [72]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Parameter {name} does not require grad")


Parameter base_model.model.embeddings.weight does not require grad
Parameter base_model.model.position_embeddings.weight does not require grad
Parameter base_model.model.blocks.0.self_attn.in_proj_weight does not require grad
Parameter base_model.model.blocks.0.self_attn.in_proj_bias does not require grad
Parameter base_model.model.blocks.0.self_attn.out_proj.base_layer.weight does not require grad
Parameter base_model.model.blocks.0.self_attn.out_proj.base_layer.bias does not require grad
Parameter base_model.model.blocks.0.linear1.weight does not require grad
Parameter base_model.model.blocks.0.linear1.bias does not require grad
Parameter base_model.model.blocks.0.linear2.weight does not require grad
Parameter base_model.model.blocks.0.linear2.bias does not require grad
Parameter base_model.model.blocks.0.norm1.weight does not require grad
Parameter base_model.model.blocks.0.norm1.bias does not require grad
Parameter base_model.model.blocks.0.norm2.weight does not require grad
Parame

In [73]:
for name, module in model.named_modules():
    if 'lora' in name:  # Or any specific identifier related to your LoRA layers
        for param in module.parameters():
            param.requires_grad = True


In [74]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Parameter {name} does not require grad")


Parameter base_model.model.embeddings.weight does not require grad
Parameter base_model.model.position_embeddings.weight does not require grad
Parameter base_model.model.blocks.0.self_attn.in_proj_weight does not require grad
Parameter base_model.model.blocks.0.self_attn.in_proj_bias does not require grad
Parameter base_model.model.blocks.0.self_attn.out_proj.base_layer.weight does not require grad
Parameter base_model.model.blocks.0.self_attn.out_proj.base_layer.bias does not require grad
Parameter base_model.model.blocks.0.linear1.weight does not require grad
Parameter base_model.model.blocks.0.linear1.bias does not require grad
Parameter base_model.model.blocks.0.linear2.weight does not require grad
Parameter base_model.model.blocks.0.linear2.bias does not require grad
Parameter base_model.model.blocks.0.norm1.weight does not require grad
Parameter base_model.model.blocks.0.norm1.bias does not require grad
Parameter base_model.model.blocks.0.norm2.weight does not require grad
Parame

In [75]:
for name, param in model.named_parameters():
    if param.requires_grad is False:
        param.requires_grad = True


In [76]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Parameter {name} does not require grad")


In [77]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

Total Parameters: 8559888


In [78]:
train_model(model, train_loader, val_loader, optimizer, config, loss_fn)

Epoch 1/5 completed with average training loss: 0.02262271709293127
Validation loss after 1 epochs: 0.02320951409637928
New best validation loss: 0.02320951409637928.


Epoch 2/5 completed with average training loss: 0.020035073770210146
Validation loss after 2 epochs: 0.023609422147274017
No improvement in validation loss. Patience counter: 1


Epoch 3/5 completed with average training loss: 0.01763782303892076
Validation loss after 3 epochs: 0.02494935318827629
No improvement in validation loss. Patience counter: 2


Epoch 4/5 completed with average training loss: 0.015539395973458886
Validation loss after 4 epochs: 0.027162861078977585
No improvement in validation loss. Patience counter: 3
Early stopping triggered.
Training completed.


In [ ]:
if not load_df:
    for input_text in input_texts_list:
        output_text = generate_text(model, custom_tokenizer, input_text, config)
        print(output_text)
        dynamic_part = f"{input_text} Story begins here:*  {''.join(output_text)}. * The story ends here"
        final_prompt = f"{step_1_static}{dynamic_part}\n{step_2}\n{step_3}"
        gemini_generated_response = evaluate_text_gemini(final_prompt)
        print(gemini_generated_response)
        gemini_generated_response = gemini_generated_response.strip()
        count += 1
    
        # print(f"{input_text}; {gemini_generated_response}")
    
        match = re.search(pattern, gemini_generated_response)
        if match:
            grammar, consistency, creativity, plot, age_group = match.groups()
            score_list.append([input_text, int(grammar), int(consistency), int(creativity), age_group])
        else:
            score_list.append([input_text, 0, 0, 0, "DNF"])
        print(f"Number of prompts appended: {count}")

In [86]:
df1 = pd.DataFrame(score_list, columns=["Input Prompt", "Grammar", "Creativity", "Consistency", "Age Group"])

In [87]:
result_req_path = config['WORKING_DIR']+'/result'

In [88]:
df1.to_pickle(result_req_path+'/'+'rating_df_LoRA'+config['MODEL_NAME']+'.pkl')

In [89]:
df1.head()

,Input Prompt,Grammar,Creativity,Consistency,Age Group
0,"In a bustling city filled with secrets, a shad...",0,0,0,DNF
1,"High in the mountains, a lone traveler braved ...",0,0,0,DNF
2,"Beneath the waves, in a hidden underwater king...",0,0,0,A
3,It was a quiet night until the distant howls b...,0,0,0,DNF
4,"In a world where dragons flew free, danger was...",0,0,0,DNF


In [90]:
df1.tail()

,Input Prompt,Grammar,Creativity,Consistency,Age Group
195,"In a land where the moon never rose, stars tol...",0,0,0,DNF
196,"In the far north, where the aurora danced, leg...",0,0,0,A
197,"At the edge of eternity, two lovers met.",0,0,0,A
198,"In a village of music, silence brought fear.",0,0,0,DNF
199,"In a world where memories could be traded, one...",0,0,0,DNF


In [91]:
model_req_path = config['WORKING_DIR']+'/model'

In [92]:
torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
                }, model_req_path+'/'+config['MODEL_NAME']+'LoRA'+'.pt')

In [93]:
df1.head(20)

,Input Prompt,Grammar,Creativity,Consistency,Age Group
0,"In a bustling city filled with secrets, a shad...",0,0,0,DNF
1,"High in the mountains, a lone traveler braved ...",0,0,0,DNF
2,"Beneath the waves, in a hidden underwater king...",0,0,0,A
3,It was a quiet night until the distant howls b...,0,0,0,DNF
4,"In a world where dragons flew free, danger was...",0,0,0,DNF
5,"On a small farm, a young girl discovered a mys...",0,0,0,DNF
6,"In a town where everyone whispered, the new st...",0,0,0,DNF
7,"Under the light of the full moon, something ma...",0,0,0,DNF
8,"In a time of peace, a hidden evil began to rise.",0,0,0,DNF
9,"Amidst the golden sands of the desert, a lost ...",0,0,0,A
